#  Webscraping avec python
# **Python**
# **TP 6 - Scraping avec BeautifulSoup**

**Inspiré de : [Cours de Python pour le Data Scientist - Lino Galiana](https://linogaliana-teaching.netlify.app/manipulation/webscraping/)**


***

[**Notions**](#notions)

1. [Balises HTML](#1)
2. [Parent et enfant](#2)
3. [Utiliser BeautifulSoup](#3)


*** 

[**Exercices**](#exercices)

- Exercice 1
- Exercice 2

***


## Exercices

## Exercice 1 : obtenir la liste des équipes de Ligue 1

Sur la page https://fr.wikipedia.org/wiki/Championnat_de_France_de_football_2019-2020, on veut scraper le tableau de la section Participants (celui avec ces colonnes : (Club, Dernière montée, Budget en M€, ...)

#### 1) Trouver le tableau


In [10]:
!pip3 show lxml
#pour l'installer :  !pip3 install lxml

Name: lxml
Version: 4.6.1
Summary: Powerful and Pythonic XML processing library combining libxml2/libxslt with the ElementTree API.
Home-page: https://lxml.de/
Author: lxml dev team
Author-email: lxml-dev@lxml.de
License: BSD
Location: /usr/local/lib/python3.9/site-packages
Requires: 
Required-by: fraddress, python-docx, svglib, yfinance


In [7]:
# content

In [4]:
import requests
from bs4 import BeautifulSoup

url_ligue_1 = "https://fr.wikipedia.org/wiki/Championnat_de_France_de_football_2019-2020"

# Requête et son contenu
response = requests.get(url_ligue_1)
content = response.content

# On parse le html
page = BeautifulSoup(content, 'html')

# On trouve le tableau
tableau_participants = page.find(
    'table', 
    class_=['wikitable', 'sortable'] # ou directement 'wikitable sortable'
)

# Dans le tableau, on trouve le fuls tbody
table_body = tableau_participants.find('tbody')

print(type(table_body))

<class 'bs4.element.Tag'>


#### 2) Récupérer chaque ligne de la table


In [21]:
# on recherche toutes les lignes du tableau avec la balise "tr"
rows = table_body.find_all('tr')

In [22]:
# Example : 
row = rows[12]

# NE pas faire : !!
#  row = row.text.split("\n")

# Il faut faire
row = [el.text.strip("\n") for el in row.find_all("td")]
print(row)

#row = [x for x in row if x != ""]
#row

['FC Nantes', '2013', '70', '12e', ' Christian Gourcuff', '2019', 'Stade de la Beaujoire - Louis Fonteneau', '35\xa0322', '51']


In [28]:
# On reproduit cette logique sur toutes les lignes
rows_cleaned = []
for row in rows:
    row_cleaned = [el.text.strip("\n") for el in row.find_all("td")]
    row_cleaned = [x for x in row_cleaned if x != ""]
    rows_cleaned.append(row_cleaned)
    
#rows_cleaned = [[x for x in row.text.split("\n") if x != ""] for row in rows] 

rows_cleaned = rows_cleaned[1:]

[len(row) for row in rows_cleaned]

[9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9]

In [33]:
# On reproduit cette logique sur toutes les lignes
l_values = []

# On récupère les entêtes avec th
values = rows[0].find_all("th")
values = [el.text.strip("\n") for el in values]
l_values.append(values)


for row in rows:
    # On récupère les valeurs dans les cellules
    values = row.find_all("td")
    values = [el.text.strip("\n") for el in values]
    if values:
        l_values.append(values)

In [35]:
print([len(row) for row in l_values])
print(l_values[0])
print(l_values[1])

[9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9]
['Club', 'Dernièremontée', 'Budget[3]en M€', 'Classement2018-2019', 'Entraîneur', 'Depuis', 'Stade', 'Capacitéen L1[4]', 'Nombrede saisonsen L1']
['Paris Saint-Germain', '1974', '637', '1er', ' Thomas Tuchel', '2018', 'Parc des Princes', '47\xa0929', '46']


In [37]:
# Deuxieme methode
from urllib import request
import ssl

# Problème pour lire depuis https avec read_html
# https://stackoverflow.com/questions/50969583/pandas-raises-ssl-certificateerror-when-using-method-read-html-for-https-resourc

url = url_ligue_1 #"https://example.com/data.html"
context = ssl._create_unverified_context()
response = request.urlopen(url, context=context)
html = response.read()

import pandas as pd
l_df = pd.read_html(html)
print(len(l_df), "tables récupérées et parsées")


34 tables récupérées et parsées


,0,1,2,3,4,5
0,Match retour,18e de la Ligue 1,Annulé,Vainqueur des barrages d'accession de la Ligue...,Stade du 18e de la Ligue 1,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
l_df[5]

,Club,Dernièremontée,Budget[3]en M€,Classement2018-2019,Entraîneur,Depuis,Stade,Capacitéen L1[4],Nombrede saisonsen L1
0,Paris Saint-Germain,1974,637,1er,Thomas Tuchel,2018,Parc des Princes,47 929,46
1,LOSC Lille,2000,120,2e,Christophe Galtier,2017,Stade Pierre-Mauroy,49 712,59
2,Olympique lyonnais,1989,310,3e,Rudi Garcia,2019,Groupama Stadium,57 206,60
3,AS Saint-Étienne,2004,100,4e,Claude Puel,2019,Stade Geoffroy-Guichard,41 965,66
4,Olympique de Marseille,1996,110,5e,André Villas-Boas,2019,Orange Vélodrome,66 226,69
5,Montpellier HSC,2009,40,6e,Michel Der Zakarian,2017,Stade de la Mosson,22 000,27
6,OGC Nice,2002,50,7e,Patrick Vieira,2018,Allianz Riviera,35 596,60
7,Stade de Reims,2018,45,8e,David Guion,2017,Stade Auguste-Delaune,20 546,35
8,Nîmes Olympique,2018,27,9e,Bernard Blaquart,2015,Stade des Costières,15 788,35
9,Stade rennais FC,1994,65,10e,Julien Stéphan,2018,Roazhon Park,29 194,62


## Exercice 2
- A partir de la suivante : [pokedex](https://pokemondb.net/pokedex/national), l'objectif est de constituer la liste de tous les pokemons
 - 1. Dans un premier temps, on voudrait la liste avec nom, numéro et types pour chaque pokémon
 - 2. Ensuite, on souhaiterait aussi garder l'url pour chaque pokemon (par exemple en utilisant l'attribut ``href``) pour pouvoir ensuite faire une requête ``GET``sur cette page (par exemple : https://pokemondb.net/pokedex/venusaur) et récupérer les données de taille et de poids

In [18]:
## Question 1 

import requests
from bs4 import BeautifulSoup
import os


url = "https://pokemondb.net/pokedex/national"

# Requête
response = requests.get(url)
content = response.content
page = BeautifulSoup(content, 'html.parser')


# Division infocard pour chaque pokemon
info_cards = page.find_all("div", class_="infocard")

l_pokemons = []

for info_card in info_cards[:15]:
   
    # On filtre sur la classe et on prend l'attribut text
    name_tag = info_card.find('a', class_="ent-name")
    name = name_tag.text
    number = info_card.find('small').text[1:]
    
    url = "https://pokemondb.net/pokedex/" + name_tag.get("href").split("/")[2]
    
    # On peut le trouver ainsi
    # types = info_card.find_all("small")[1]
    #print(types)
    
    # On peut aussi utiliser
    types  = info_card.find('small').find_next("small")#.text
    
    
    #print(types)
    types = [t.strip() for t in types.text.split(" · ")]
    
    d = {"name":name, "number":number, "types":types, "url": url}
   
    l_pokemons.append(d)
## Question 1 (partielle, il n'y a pas les types)


In [20]:
# from pprint import pprint
# pprint(l_pokemons)

In [21]:
l_pokemons[0]

{'name': 'Bulbasaur',
 'number': '0001',
 'types': ['Grass', 'Poison'],
 'url': 'https://pokemondb.net/pokedex/bulbasaur'}

In [22]:
for pokemon in l_pokemons[:2]:
    url = pokemon["url"]
    print(url)
    
    # Requête
    response = requests.get(url)
    content = response.content
    page = BeautifulSoup(content, 'html.parser')
    
    # On récupère les nom des variables (en-têtes mais à gauche dans cette table)
    ths = page.find("table", class_="vitals-table").find_all("th")
    
    # On utilise find_next après avoir trouvé l'entête
    # In this order
    data = [th.find_next().text for th in ths if th.text in ["Height", "Weight"]]
    print(data)
    
    height = data[0].split("(")[0].replace("\xa0", " ").strip()
    weight = data[1].split("(")[0].replace("\xa0", " ").strip()

    pokemon["height"] = height
    pokemon["weight"] = weight

https://pokemondb.net/pokedex/bulbasaur
['0.7\xa0m (2′04″)', '6.9\xa0kg (15.2\xa0lbs)']
https://pokemondb.net/pokedex/ivysaur
['1.0\xa0m (3′03″)', '13.0\xa0kg (28.7\xa0lbs)']
